In [ ]:
%matplotlib inline

import cv2
import numpy as np
import pandas as pd
from os import listdir

from utils import imshow

pd.options.display.html.border = 0
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
data_path = "./data/"
cars = [cv2.imread(data_path+img, 1) for img in listdir(data_path)]
imshow(cars[0])

In [ ]:
def median_filter(imgs: list[np.ndarray]) -> np.ndarray:
    imgs_stacked = np.stack(imgs, axis=2)
    imgs_gray = [cv2.cvtColor(car, cv2.COLOR_BGR2GRAY) for car in imgs]
    imgs_gray_stacked = np.stack(imgs_gray,axis=2)
    imgs_median_arg = np.argsort(imgs_gray_stacked,axis=2)[:,:,1]

    m, n, _ = imgs[0].shape
    i, j = np.indices((m, n))
    imgs_color = imgs_stacked[i, j, imgs_median_arg[i, j]]
    
    return imgs_color
        

In [ ]:
img_median = median_filter(cars)

imshow(img_median)

The result of the median filter is very good, however there's some car artifacts on the right where, there are cars in 2 pictures.

In [ ]:
def subtract_imgs(img_1: np.ndarray, img_2: np.ndarray) -> np.ndarray:
    img_1_gray = cv2.cvtColor(img_1, cv2.COLOR_BGR2GRAY)
    img_2_gray = cv2.cvtColor(img_2, cv2.COLOR_BGR2GRAY)
    img_diff = cv2.absdiff(img_1_gray, img_2_gray)
    return img_diff

In [ ]:
img_sub = subtract_imgs(cars[0], cars[1])
imshow(img_sub)

In [ ]:
img_thresh = cv2.threshold(img_sub, 35, 255, cv2.THRESH_BINARY)[1]
imshow(img_thresh)

In [ ]:
img_space_closed = cv2.morphologyEx(img_thresh, cv2.MORPH_CLOSE, np.ones((5,5),np.uint8))
imshow(img_space_closed)

In [ ]:
img_masked = cv2.bitwise_and(cars[2], cars[2], mask=img_space_closed)
imshow(img_masked)

In [ ]:
img_thresh_stacked = np.stack([img_space_closed]*3, axis=2)
img_median_masked = np.where(img_thresh_stacked == 255, img_masked, img_median)
imshow(img_median_masked)